In [2]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
	os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
	os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
	os.system("pip install Counter")

if importlib.util.find_spec("openpyxl") is None:
	os.system("pip install openpyxl")


In [3]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('../.env', override=True)

    JiraHost = os.getenv('SECRETS_HOST')
    ConfluenceHost = os.getenv('SECRETS_CONFLUENCE')
    Username = os.getenv('SECRETS_USERNAME')
    Password = os.getenv('SECRETS_PASSWORD')
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://jiradev.budgetdirect.com.au/'

'Confluence Host: https://confluence.budgetdirect.com.au/'

In [4]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings
import json

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0"
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def flatten_reduce_lambda(frm):
	try:
		return list(reduce(lambda x, y: x + y, frm, []))
	except:
		return list(reduce(lambda x, y: x + y, [frm], []))

def most_frequent(List):
	try:
		c = Counter(List)
		most_common = [key for key, _ in c.most_common(5)]
		return most_common
	except:
		return ""

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnUpdateBoardAdmins(boardId, boardAdmins, boardGroups):
	boardAdminsData = {}
	boardAdminsData['userKeys'] = boardAdmins
	boardAdminsData['groupKeys'] = boardGroups
	data = {}
	data['id'] = boardId
	data['boardAdmins'] = boardAdminsData
	url = "/rest/greenhopper/1.0/rapidviewconfig/boardadmins"
	headers = fnGetDefaultHeaders()
	response = requests.put(JiraHost + url, headers = headers, data = json.dumps(data), verify=False)
	return response.json()

def fnUpdateFilterOwner(filterId, jql, name, description, filterOwner):
	data = {}
	data['jql'] = jql
	data['name'] = name
	data['description'] = description
	data['owner'] = {}
	data['owner']['key'] = filterOwner
	url = "/rest/api/2/filter/" + str(filterId)
	headers = fnGetDefaultHeaders()
	response = requests.put(JiraHost + url, headers = headers, data = json.dumps(data), verify=False)
	return response.json()

def fnGetFilterDetails(filterId):
	url = "/rest/api/2/filter/" + str(filterId)
	headers = fnGetDefaultHeaders()
	response = requests.get(JiraHost + url, headers = headers, verify=False)
	return response.json()

def fnBoards():
    def ApiCall(startAt) :
        url = "/rest/greenhopper/1.0/rapidviews/viewsData"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt
        }
        response = requests.get(JiraHost + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    return values

def fnUsers():
     #/rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt=7000
	def ApiCall(startAt) :
		url = "/rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt=" + str(startAt)
		headers = fnGetDefaultHeaders()
		response = requests.get(JiraHost + url, headers = headers, verify=False)
		return response.json()
	return pd.DataFrame(flatten_reduce_lambda([ApiCall(i * 1000) for i in range(8)]))

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df



In [5]:
users = fnUsers()

display(users.head())

,self,key,name,emailAddress,avatarUrls,displayName,active,deleted,timeZone,locale
0,https://jiradev.budgetdirect.com.au/rest/api/2...,aadelakun,aadelakun,Ade.Adelakun@aihco.com.au,{'48x48': 'https://jiradev.budgetdirect.com.au...,Ade Adelakun,False,False,Australia/Brisbane,en_US
1,https://jiradev.budgetdirect.com.au/rest/api/2...,JIRAUSER32069,aalang,aaron.lang@autogeneral.com.au,{'48x48': 'https://jiradev.budgetdirect.com.au...,Aaron Lang,True,False,Australia/Brisbane,en_US
2,https://jiradev.budgetdirect.com.au/rest/api/2...,aalburydor,aalburydor,andrew.alburydor@autogeneral.com.au,{'48x48': 'https://jiradev.budgetdirect.com.au...,Andrew Albury-Dor [X],False,False,Australia/Brisbane,en_US
3,https://jiradev.budgetdirect.com.au/rest/api/2...,JIRAUSER34432,aalkhanji,anas.alkhanji@autogeneral.com.au,{'48x48': 'https://jiradev.budgetdirect.com.au...,Anas Alkhanji,True,False,Australia/Brisbane,en_US
4,https://jiradev.budgetdirect.com.au/rest/api/2...,JIRAUSER33748,aalmaradni,ahmad.almaradni@autogeneral.com.au,{'48x48': 'https://jiradev.budgetdirect.com.au...,Ahmad Almaradni,True,False,Australia/Brisbane,en_US


In [6]:
df = fnBoards()

df = df.explode("views")
df = df.drop(["total", "globalConfig"], axis=1)
df = df.expand("views", [], None, False)
df = df.drop(["canEdit", "sprintSupportEnabled"], axis=1)
df = df.expand("filter", [], "Prefix", False)
df = df.drop(["filter canEdit", "filter isOrderedByRank", "filter permissionEntries", "filter canBeFixed"], axis=1)
df = df.expand("filter owner", [], "Prefix", False)
df = df.drop(["filter owner renderedLink"], axis=1)

df2 = df.expand("boardAdmins", [], "Prefix", False)
df2 = df2.expand("boardAdmins userKeys", [], "Prefix", False)
for i in range(6): df2 = df2.expand("boardAdmins userKeys " + str(i), [], "Prefix", False)

df2 = df2.expand("boardAdmins groupKeys", [], "Prefix", False)
for i in range(2): df2 = df2.expand("boardAdmins groupKeys " + str(i), [], "Prefix", False)

try:
	df2['boardAdmins groups'] = df2[["boardAdmins groupKeys " + str(i) + " key" for i in range(1)]].apply(lambda row: '|'.join(filter(lambda value: str(value).upper() != "NAN", row.values.astype(str))), axis=1)
	df['boardAdmin Groups'] = df2["boardAdmins groups"]
except:
	pass

def linkRel(row):
	soup = BeautifulSoup(' '.join(row.values.astype(str)), 'html.parser').findAll('a')
	return "|".join(a.get('rel')[0] if a != None else "" for a in soup) if soup != None else ""

cols = ["boardAdmins userKeys " + str(i) + " displayName" for i in range(6)]
df['boardAdminUsernames'] = df2[cols].apply(linkRel, axis=1)


df['newAdmins'] = ""
df['newFilterOwner'] = ""
df = df.reset_index()  # make sure indexes pair with number of rows

## This is not correct pandas but i couldn't make it work otherwise!
for index, row in df.iterrows():
	newAdmins = []
	requiresDefault = False
	for username in row['boardAdminUsernames'].split("|"):
		userRecord = users.loc[users['name'] == username]
		if userRecord.empty:
			continue
		if userRecord['active'].bool():
			newAdmins.append(username)
	if len(newAdmins) == 0: ## & requiresDefault:
		df.at[index,'newAdmins'] = "pogliani"
	else:
		df.at[index,'newAdmins'] = '|'.join(newAdmins)

	filterOwner = row['filter owner userName']
	userRecord = users.loc[users['name'] == filterOwner]
	if userRecord.empty:
		continue
	if userRecord['active'].bool():
		df.at[index,'newFilterOwner'] = filterOwner
	else:
		df.at[index,'newFilterOwner'] = "pogliani"

df['requiresUpdateToBoard'] = df['newAdmins'] != df['boardAdminUsernames']
df['requiresUpdateToFilter'] = df['newFilterOwner'] != df["filter owner userName"]

#df.to_excel('../ListBoardOwners.xlsx', index=False)

display(df)

,index,id,name,boardAdmins,filter id,filter name,filter query,filter owner userName,filter owner displayName,boardAdmin Groups,boardAdminUsernames,newAdmins,newFilterOwner,requiresUpdateToBoard,requiresUpdateToFilter
0,0,1552,.,"{'userKeys': [{'key': 'JIRAUSER33768', 'displa...",46996,Filter for Cheaper Elsewhere,project = DSD ORDER BY Rank ASC,vhampsonti,Vere Hampson - Tindale,,vhampsonti,vhampsonti,vhampsonti,False,False
1,0,1331,A&BD AGPs,"{'userKeys': [{'key': 'gwebb', 'displayName': ...",43449,All Alliances AGPs,project = AGP AND issuetype = Opportunity AND ...,gwebb,Gary Webb,,gwebb,gwebb,gwebb,False,False
2,0,1260,A&DB Requirement Elaboration Board,"{'userKeys': [{'key': 'JIRAUSER31636', 'displa...",43640,Filter for A&DB Requirement Elaboration Board,project = DLITE ORDER BY Rank ASC,pogliani,Paulo Ogliani,,pogliani,pogliani,pogliani,False,False
3,0,1551,ACAD2023 board,"{'userKeys': [{'key': 'JIRAUSER33505', 'displa...",46763,Filter for ACAD2023 board,project = ACAD2023 ORDER BY Rank ASC,mwalton,Mike Walton,,mwalton|jfrew,mwalton|jfrew,mwalton,False,False
4,0,552,Africa Promo Initiative,"{'userKeys': [{'key': 'JIRAUSER31636', 'displa...",33183,Africa_promo_filter,labels = Africa_promo_initiative,ijohnson,Isaac Johnson [X],,pogliani,pogliani,pogliani,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,0,1200,WOPS Kanban,"{'userKeys': [{'key': 'JIRAUSER31409', 'displa...",43158,Filter for WOPS Kanban,project = WOPS ORDER BY Rank ASC,apreethasi,Asvin Preetha Sivadasan,,apreethasi|ismith,apreethasi|ismith,apreethasi,False,False
872,0,1148,WOPS: Sprint Board,"{'userKeys': [{'key': 'JIRAUSER31409', 'displa...",42886,WOPS: Sprint Board Filter,"project = WOPS OR project in (CSS, RAD, TID, U...",apreethasi,Asvin Preetha Sivadasan,,apreethasi|cthear|ismith|jverma|mabryan|sbayat,apreethasi|cthear|ismith|jverma|mabryan|sbayat,apreethasi,False,False
873,0,1086,Workflow - Lily Neville,"{'userKeys': [{'key': 'lneville', 'displayName...",42569,Filter for Workflow - Lily Neville,"project in (MMAD, MDNC, MDEC, MDDA) AND issuet...",lneville,Lily Neville,,lneville,lneville,lneville,False,False
874,0,589,Workflow Data & Doco Summary,"{'userKeys': [{'key': 'smackay', 'displayName'...",34387,Workflow Data & Doco Epics,"project = ""Workflow, Data, Doco"" AND issuetype...",smackay,Sandy Mackay,,smackay,smackay,smackay,False,False


In [7]:
#! Remove Inactive Board Users

removeInactiveBoardUsersDf = df[df['requiresUpdateToBoard']]
removeInactiveBoardUsersDf = removeInactiveBoardUsersDf[removeInactiveBoardUsersDf['requiresUpdateToBoard']]
removeInactiveBoardUsersDf = removeInactiveBoardUsersDf.drop(["boardAdmins", "name", "filter id", "filter owner userName", "filter name", "filter query", "filter owner displayName", "newFilterOwner", "requiresUpdateToBoard", "requiresUpdateToFilter"], axis=1)

display(removeInactiveBoardUsersDf)

errors = 0
updated = 0
for index, row in removeInactiveBoardUsersDf.iterrows():

	admins = []
	for username in row['newAdmins'].split("|"):
		userRecord = users.loc[users['name'] == username]
		admins.append(userRecord['key'].values[0])

	display((row['id'], admins, row['boardAdmin Groups'].split("|") if len(row['boardAdmin Groups']) > 0 else []))

	result = fnUpdateBoardAdmins(row['id'], admins, row['boardAdmin Groups'].split("|") if len(row['boardAdmin Groups']) > 0 else [])
	if ('errorMessages' in result):
		display((result, row['id'], admins, row['boardAdmin Groups'].split("|") if len(row['boardAdmin Groups']) > 0 else []))
		errors = errors + 1
	else:
		updated = updated + 1

if errors > 0 :
    print (f"Errors {errors} out of {len(removeInactiveBoardUsersDf.index)}")
else:
	if updated > 0:
		print (f"Updated {updated} out of {len(removeInactiveBoardUsersDf.index)}")
	else:
		print ("Clean!")


,index,id,boardAdmin Groups,boardAdminUsernames,newAdmins


Clean!


In [8]:
#! Update Filter Users
import json

changeFilterOwner = df[df['requiresUpdateToFilter']]
#changeFilterOwner = changeFilterOwner[changeFilterOwner['filter id'].isin([38941])]
changeFilterOwner = changeFilterOwner.drop(["boardAdmins", "name", "id", "boardAdminUsernames", "requiresUpdateToBoard", "newAdmins", "requiresUpdateToFilter"], axis=1)
try:
	changeFilterOwner = changeFilterOwner.drop(["boardAdmin Groups"], axis=1)
except:
	pass

display(changeFilterOwner)

errors = 0
updated = 0
for index, row in changeFilterOwner.iterrows():
	try:
		filter = fnGetFilterDetails(row['filter id'])
		result = fnUpdateFilterOwner(row['filter id'], row['filter query'], row['filter name'], filter['description'], row['newFilterOwner'])
		if ('errorMessages' in result):
			display(result.errorMessage)
			errors = errors + 1
		else:
			updated = updated + 1
	except:
		display("Error updating", (row['filter id'], row['filter query'], row['filter name'], row['newFilterOwner']))
		errors = errors + 1

if errors > 0 :
    print (f"Errors {errors} out of {len(changeFilterOwner.index)}")

if updated > 0:
	print (f"Updated {updated} out of {len(changeFilterOwner.index)}")
else:
	print ("Clean!")


,index,filter id,filter name,filter query,filter owner userName,filter owner displayName,newFilterOwner
4,0,33183,Africa_promo_filter,labels = Africa_promo_initiative,ijohnson,Isaac Johnson [X],pogliani
5,0,37433,Filter for AGGI Spirit Board,project = AGQL ORDER BY Rank ASC,ghari,Govind Hari [X],pogliani
7,0,39081,Filter for Change Delivery Board,project = CHDLV ORDER BY Rank ASC,ghari,Govind Hari [X],pogliani
8,0,44493,Filter for System Support - AGP Initiatives,project = AGP AND resolution = Unresolved AND ...,rewalsh,Reece Walsh [X],pogliani
9,0,38941,AGP: Large or Complex,"project = AGP AND ""Portfolio Change Type"" = ""L...",mhastie,Melody Hastie [X],pogliani
...,...,...,...,...,...,...,...
845,0,41385,Filter for TT board,project = TT ORDER BY Rank ASC,mhastie,Melody Hastie [X],pogliani
846,0,41399,Filter for TT Sprint 1,project = TT ORDER BY Rank ASC,gboon,Gareth Boon [X],pogliani
849,0,42553,Filter for UAT Defect Board,project = UATT,fhowell,Fiona Howell [X],pogliani
850,0,42553,Filter for UAT Defect Board,project = UATT,fhowell,Fiona Howell [X],pogliani


(33183,
 'labels = Africa_promo_initiative',
 'Africa_promo_filter',
 'unknown',
 'pogliani')

(37433,
 'project = AGQL ORDER BY Rank ASC',
 'Filter for AGGI Spirit Board',
 'unknown',
 'pogliani')

(44493,
 'project = AGP AND resolution = Unresolved AND "Delivery Teams" in ("Support Optimisation", "Support Triage") ORDER BY Rank ASC',
 'Filter for System Support - AGP Initiatives',
 'unknown',
 'pogliani')

(38941,
 'project = AGP AND "Portfolio Change Type" = "Large or Complex" ORDER BY Rank ASC',
 'AGP: Large or Complex',
 'unknown',
 'pogliani')

...

(41399,
 'project = TT ORDER BY Rank ASC',
 'Filter for TT Sprint 1',
 'unknown',
 'pogliani')

(42553, 'project = UATT', 'Filter for UAT Defect Board', 'unknown', 'pogliani')

(42553, 'project = UATT', 'Filter for UAT Defect Board', 'unknown', 'pogliani')

(18255,
 'issuetype = "Creative Design" AND status not in (Closed, Resolved) AND project not in ("CtM Marketing")',
 'All Design Tickets',
 'unknown',
 'pogliani')

Errors 323 out of 446
Updated 123 out of 446
